# Proyecto Individual 1 (MLOPs): Sistema de Recomendación de Películas

## Modelado

In [95]:
''' Importamos las librerías necesarias'''

import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [96]:
''' Importamos el dataset'''

movies_df = pd.read_csv('C:/Users/Luis/Documents/Universidad-Trabajo/SoyHenry/LABS/Proyecto_1/Datasets/Movies_limpio.csv')


In [97]:
''' Antes de realizar el modelado, haremos algunos cambios en el conjunto de datos para reducir la cantidad de datos
    y poder correr los diferentes algoritmos. Esto no se realizó antes para que el dataset pueda funcionar en las
    diferentes funciones de la api'''

'''Eliminamos las películas que no hayan sido lanzadas, es decir, que su status sea diferente a released'''

movies_df = movies_df[movies_df['status'] == 'Released']

In [98]:
''' Del EDA observamos que hay outliers en la columna runtime. En este caso dejaremos los valores que sean mayores
    a 0 minutos y menores a 500 minutos, considerando que en promedio una película dura entre 60 a 180 minutos'''

movies_df = movies_df[(movies_df['runtime'] > 0) & (movies_df['runtime'] <= 500)]

In [99]:
''' Del EDA también observamos que no es hasta el año 1920 que se comienzan a producir mayor cantidad de películas.
    Por esto se dejarán solo las películas que se hayan estrenado a partir de 1920.'''

movies_df = movies_df[movies_df['release_year'] >= 1920]

In [100]:
''' Considerando que las películas más votadas por los fans son las más populares, pudiendo observarse una correlación
    entre las variables en el heatmap y pairplot realizado en el EDA, se eliminarán los registros que tengan menos de 150
    en la columna vote_count.'''

movies_df = movies_df[movies_df['vote_count'] >= 150]

In [101]:
''' Como se hará un procesamiento de lenguaje natural, se utilizará un parámetro en el modelo llamado stopwords
    que depende del idioma, se trabajará con inglés por ser el lenguaje mayoritario. Por esto, se eliminan aquellas
    películas cuyo lenguaje original no sea inglés'''

movies_df = movies_df[movies_df['original_language'] == 'en']

In [102]:
movies_df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'revenue', 'runtime', 'status',
       'title', 'vote_average', 'vote_count', 'cast', 'name_collection',
       'directors', 'release_year', 'return', 'title_list', 'union'],
      dtype='object')

In [103]:
''' Reseteamos los índices y borramos las columna que son innecesarias para el modelo'''

movies_df = movies_df.reset_index()
movies_df.drop(columns = ['index', 'budget', 'genres', 'original_language', 'overview', 'popularity', 'production_companies', 'release_date', 'revenue', 'runtime', 'status', 'vote_average', 'vote_count', 'cast', 'name_collection', 'directors', 'release_year', 'return', 'title_list'], inplace = True)

In [104]:
''' Utilizaremos a librería nltk, especificamente el módulo de stemming para realizar la derivación
    de la columna unión'''

stemmer = PorterStemmer()

''' Creamos una función que realizará la derivación a toda la columna'''

def stemming(column):
    union_list = []
    for l in column:
        union_list.append(stemmer.stem(l))
    
    return ' '.join(union_list)

In [105]:
''' Normalizamos la columna union, para reducir la cantidad de palabras y quedarnos con las más importantes'''

movies_df['union'] = movies_df['union'].apply(lambda x: re.sub("[^a-zA-Z]"," ",str(x))) # reemplazamos los caracteres que no sean letras por espacios
movies_df['union'] = movies_df['union'].apply(lambda x: x.lower()) # Llevamos todo a minúsculas
movies_df['union'] = movies_df['union'].apply(lambda x: nltk.word_tokenize(x)) # Tokenizamos

''' Aplicamos la función stemming'''

movies_df['union'] = movies_df['union'].apply(stemming)

In [106]:
''' Creamos un nuevo archivo csv a partir del movies_df. Este archivo será el usado en la función de recomendación
    de la API.'''

movies_df.to_csv('C:/Users/Luis/Documents/Universidad-Trabajo/SoyHenry/LABS/Proyecto_1/Datasets/Movies_modelo.csv')

In [107]:
''' Vectorizamos la columna union y se remueven las stopwords'''

cv = TfidfVectorizer(max_features = 2000, stop_words = 'english')

vector = cv.fit_transform(movies_df['union']).toarray()

In [108]:
''' Se utiliza el módulo cosine_similarity de sklearn, para establecer las distancias entre los diferentes
    valores del vector creado, es decir, entre los valores de la columna union'''

similarity = cosine_similarity(vector)

c:\Users\Luis\Documents\Universidad-Trabajo\SoyHenry\LABS\Proyecto_1\env_PI_1\Lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
